In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from preprocessor import Preprocessor
import random
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from itertools import combinations

In [2]:
train_preprocessor = Preprocessor()

raw_data = pd.read_csv("../../data/wikipedia.csv")
# clean out where text is NaN
raw_data = raw_data[raw_data.text.notna()]

In [33]:
x = []
y = []
for row in raw_data.text:
    # get words TODO only relevant words
    words = train_preprocessor.normalize_words(row)

    for (a, b) in combinations(words,2):
        x.append(a)
        y.append(b)

In [36]:
len(x)

53772

In [37]:
features = tf.constant(x, shape=(1,len(x),300))

labels = tf.constant(y, shape=(1,len(y),300))

#train_dataset = tf.data.Dataset.from_tensor_slices((features,labels))
features_dataset = tf.data.Dataset.from_tensor_slices(features)
labels_dataset = tf.data.Dataset.from_tensor_slices(labels)

train_dataset = tf.data.Dataset.zip((features_dataset, labels_dataset))

In [21]:
train_dataset

<ZipDataset shapes: ((91, 300), (91, 300)), types: (tf.float32, tf.float32)>

In [38]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(100, activation=tf.nn.relu, input_shape=(300,)),
  # tf.keras.layers.Dense(1000, activation=tf.nn.relu),
  tf.keras.layers.Dense(100, activation=tf.nn.relu),
  tf.keras.layers.Dense(300)
])

In [27]:
loss_object = tf.keras.losses.MeanSquaredError()

# def get_random_result(label, labels, predictions, predictions_cache):
#   label = label[0]
#   # TODO: Check if the index of the x value we are testing is not exactly the same as the one we are returning as y
#   if(label not in predictions_cache):
#     predictions_cache[label] = []
#     for idx2, value in enumerate(labels.numpy()):
#         if(value == label):
#             predictions_cache[label].append(predictions[idx2])
#   return predictions_cache[label][random.randint(0,len(predictions_cache[label])-1)]

# def get_new_y(predictions, labels):
#   predictions_cache = {}
#   new_y = []
#   for idx, val in enumerate(predictions.numpy()):
#       label = labels[idx].numpy()
#       new_y.append(get_random_result(label, labels, predictions, predictions_cache))
#   return new_y

def loss(model, x, y, training):
  # training=training is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  y_ = model(x, training=training)
  #new_y = get_new_y(y_, y)

  return loss_object(y_true=y, y_pred=y_)

def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets, training=True)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [28]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

In [40]:
## Note: Rerunning this cell uses the same model variables

# Keep results for plotting
train_loss_results = []
train_accuracy_results = []
num_epochs = 100
import time
for epoch in range(num_epochs):
  start = time.time()
  epoch_loss_avg = tf.keras.metrics.Mean()
  epoch_accuracy = tf.keras.metrics.MeanAbsoluteError()

  # Training loop - using batches of 32
  for x, y in train_dataset:
    # Optimize the model
    loss_value, grads = grad(model, x, y)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Track progress
    epoch_loss_avg.update_state(loss_value)  # Add current batch loss
    # Compare predicted label to actual label
    # training=True is needed only if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(x, training=True)
    #new_y = get_new_y(predictions, y)
    epoch_accuracy.update_state(y, predictions)

  # End epoch
  train_loss_results.append(epoch_loss_avg.result())
  train_accuracy_results.append(epoch_accuracy.result())
  end = time.time()
  print("Epoch {:03d}: Loss: {:.3f}, MAE: {:.3}, time: {:.8f}".format(epoch,epoch_loss_avg.result(),epoch_accuracy.result(), end - start))

Epoch 000: Loss: 11.165, MAE: 2.1, time: 1.02741194
Epoch 001: Loss: 11.129, MAE: 2.1, time: 0.84673786
Epoch 002: Loss: 11.097, MAE: 2.09, time: 0.84185958
Epoch 003: Loss: 11.069, MAE: 2.09, time: 0.84808397
Epoch 004: Loss: 11.043, MAE: 2.08, time: 1.04802966
Epoch 005: Loss: 11.019, MAE: 2.08, time: 0.83689475
Epoch 006: Loss: 10.998, MAE: 2.07, time: 0.84041691
Epoch 007: Loss: 10.979, MAE: 2.07, time: 0.79074359
Epoch 008: Loss: 10.961, MAE: 2.07, time: 0.75465703
Epoch 009: Loss: 10.944, MAE: 2.06, time: 0.93554473
Epoch 010: Loss: 10.928, MAE: 2.06, time: 0.92920852
Epoch 011: Loss: 10.914, MAE: 2.06, time: 0.85919714
Epoch 012: Loss: 10.900, MAE: 2.05, time: 0.87337899
Epoch 013: Loss: 10.888, MAE: 2.05, time: 0.78520656
Epoch 014: Loss: 10.876, MAE: 2.05, time: 0.82922959
Epoch 015: Loss: 10.864, MAE: 2.05, time: 0.88866067
Epoch 016: Loss: 10.854, MAE: 2.05, time: 0.75307798
Epoch 017: Loss: 10.844, MAE: 2.04, time: 0.75432181
Epoch 018: Loss: 10.834, MAE: 2.04, time: 0.7835

In [38]:
a = train_preprocessor.vectorize("Teiglinge nach der Stückgare")
b = train_preprocessor.vectorize("Das Gerät wird gleichzeitig gezogen oder vom Traktor getragen und über die Zapfwelle oder hydraulisch angetrieben")
c = train_preprocessor.vectorize("Portionsstücke werden abgeteilt; der Teig wird „ausgebrochen“")
d = train_preprocessor.vectorize("Für aktuelle Landtechnik sind die führenden Zeitschriften:")

a = train_preprocessor.vectorize("Brot ist ein leckeres im Ofen gebackenes Rezept")
b = train_preprocessor.vectorize("In der Landwirdschaft gibt es viele Traktoren")
c = train_preprocessor.vectorize("Lecker sind z.B. Roggenbrot oder Walnussbrötchen")
d = train_preprocessor.vectorize("Viele Bauern leben neben ihren Garagen um leicht ihre Zylinder reinigen zu können")

a = train_preprocessor.vectorize("Der Brauch, Brot und Salz etwa zum Bezug einer neuen Wohnung oder zur Hochzeit zu schenken, soll Wohlstand sichern.")
b = train_preprocessor.vectorize("Die zunehmende Sammelleidenschaft für historische Landmaschinen in der Bevölkerung zog auch die Ausgabe diverser Fachzeitschriften nach sich, die nach langer Dominanz klassischer Vereinsnachrichten den Markt um professionelle journalistische Produkte bereicherten. Diese Publikationen sind meist im Bahnhofsbuchhandel erhältlich. Sie informieren im Zeitschriftenformat und in Farbe über Restaurierungsprojekte, Vereinsaktivitäten, Schleppertreffen und vieles mehr.")
c = train_preprocessor.vectorize("„Brot wird ganz oder teilweise aus Getreide und/oder Getreideerzeugnissen, meist nach Zugabe von Flüssigkeit, sowie von anderen Lebensmitteln (z. B. Leguminosen-, Kartoffelerzeugnisse) in der Regel durch Kneten, Formen, Lockern, Backen oder Heißextrudieren des Brotteiges hergestellt. Brot enthält weniger als 10 Gewichtsteile Fett und/oder Zuckerarten auf 90 Gewichtsteile Getreide und/oder Getreideerzeugnisse.")
d = train_preprocessor.vectorize("Um Unfälle mit landwirtschaftlichen Zugmaschinen abzuschwächen oder ganz zu vermeiden, empfiehlt die UDV eine bessere Schulung junger Traktorfahrer. Ferner sollten alle Verkehrsteilnehmer besser über das besondere Unfallrisiko mit Schleppern aufgeklärt werden. Das Signalbild von Traktoren mit und ohne Anhänger müsse verbessert werden, beispielsweise durch die Zulassung von Rundum-Leuchten, Reflexfolien, Begrenzungsleuchten und Konturmarkierungen. Heckleuchten und Blinker sollten sowohl größer als auch stabiler sein.")

trained = model(tf.constant([a,b,c,d]), training=False)

diff1 = np.linalg.norm(trained[0]-trained[2])
diff2 = np.linalg.norm(trained[1]-trained[3])
diff3 = np.linalg.norm(trained[0]-trained[3])
diff4 = np.linalg.norm(trained[1]-trained[2])

print("Should be low")
print(diff1)
print(diff2)

print("Should be higher")
print(diff3)
print(diff4)

Should be low
0.7402869
0.4161326
Should be higher
0.6740105
0.5865265


In [4]:
a = train_preprocessor.vectorize("Der Brauch, Brot und Salz etwa zum Bezug einer neuen Wohnung oder zur Hochzeit zu schenken, soll Wohlstand sichern.")
b = train_preprocessor.vectorize("Die zunehmende Sammelleidenschaft für historische Landmaschinen in der Bevölkerung zog auch die Ausgabe diverser Fachzeitschriften nach sich, die nach langer Dominanz klassischer Vereinsnachrichten den Markt um professionelle journalistische Produkte bereicherten. Diese Publikationen sind meist im Bahnhofsbuchhandel erhältlich. Sie informieren im Zeitschriftenformat und in Farbe über Restaurierungsprojekte, Vereinsaktivitäten, Schleppertreffen und vieles mehr.")
c = train_preprocessor.vectorize("„Brot wird ganz oder teilweise aus Getreide und/oder Getreideerzeugnissen, meist nach Zugabe von Flüssigkeit, sowie von anderen Lebensmitteln (z. B. Leguminosen-, Kartoffelerzeugnisse) in der Regel durch Kneten, Formen, Lockern, Backen oder Heißextrudieren des Brotteiges hergestellt. Brot enthält weniger als 10 Gewichtsteile Fett und/oder Zuckerarten auf 90 Gewichtsteile Getreide und/oder Getreideerzeugnisse.")
d = train_preprocessor.vectorize("Um Unfälle mit landwirtschaftlichen Zugmaschinen abzuschwächen oder ganz zu vermeiden, empfiehlt die UDV eine bessere Schulung junger Traktorfahrer. Ferner sollten alle Verkehrsteilnehmer besser über das besondere Unfallrisiko mit Schleppern aufgeklärt werden. Das Signalbild von Traktoren mit und ohne Anhänger müsse verbessert werden, beispielsweise durch die Zulassung von Rundum-Leuchten, Reflexfolien, Begrenzungsleuchten und Konturmarkierungen. Heckleuchten und Blinker sollten sowohl größer als auch stabiler sein.")

In [25]:
original_diff1 = np.linalg.norm(a-c)
original_diff2 = np.linalg.norm(b-d)
original_diff3 = np.linalg.norm(a-d)
original_diff4 = np.linalg.norm(b-c)

max_val = max(original_diff1,original_diff2,original_diff3,original_diff4)

print("Original distances\n")
print("Should be low")
print(((original_diff1)/max_val))
print(((original_diff2)/max_val))

print("Should be higher")
print(((original_diff3)/max_val))
print(((original_diff4)/max_val))

Original distances

Should be low
1.0
0.54048246
Should be higher
0.6637904
0.85073495


In [15]:
_max_val = 0.7402869

print('Should be low')
print((0.7402869/_max_val))
print((0.4161326/_max_val))
print('Should be higher')
print((0.6740105/_max_val))
print((0.5865265/_max_val))

Should be low
1.0
0.5621234145842646
Should be higher
0.9104720075419408
0.7922962029991345
